In [121]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


data/transactions_test.csv
data/transactions_train.csv
data/train_target.csv
data/small_group_description.csv
data/test.csv


In [122]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [123]:
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 9)

Populating the interactive namespace from numpy and matplotlib


In [124]:
transactions_train=pd.read_csv('data/transactions_train.csv')
train_target=pd.read_csv('data/train_target.csv')
print(transactions_train.head(),'\n',train_target.head())

   client_id  trans_date  small_group  amount_rur
0      33172           6            4      71.463
1      33172           6           35      45.017
2      33172           8           11      13.887
3      33172           9           11      15.983
4      33172          10           11      21.341 
    client_id  bins
0      24662     2
1       1046     0
2      34089     2
3      34848     1
4      47076     3


In [125]:
transactions_train['tr_month'] = (transactions_train['trans_date']/30).astype('int')


In [126]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()
agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


In [127]:
counter_df_train=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_counts_train=cat_counts_train.fillna(0)
cat_counts_train.columns=['small_group_'+str(i) for i in cat_counts_train.columns]
cat_counts_train.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,...,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
client_id,,,,,,,,,,,,,,,,,,,,,
4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,cat_counts_train.reset_index(),on='client_id')
train.head()

,client_id,bins,sum,mean,std,min,max,small_group_0,small_group_1,small_group_2,...,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
transactions_test=pd.read_csv('data/transactions_test.csv')
transactions_test['tr_month'] = (transactions_test['trans_date']/30).astype('int')
test_id=pd.read_csv('data/test.csv')

In [130]:
transactions_train['trans_date'].max()

729

In [131]:
agg_features_test=transactions_test.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()
counter_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()
cat_counts_test=counter_df_test.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')
cat_counts_test=cat_counts_test.fillna(0)
cat_counts_test.columns=['small_group_'+str(i) for i in cat_counts_test.columns]
cat_counts_test.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,...,small_group_192,small_group_193,small_group_194,small_group_195,small_group_196,small_group_197,small_group_198,small_group_200,small_group_201,small_group_202
client_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,226.0,1.0,36.0,9.0,0.0,0.0,0.0,2.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.0,326.0,0.0,40.0,56.0,0.0,0.0,0.0,0.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.0,242.0,1.0,50.0,48.0,4.0,0.0,6.0,1.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,156.0,83.0,48.0,31.0,2.0,0.0,1.0,2.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,16.0,398.0,1.0,23.0,25.0,0.0,0.0,0.0,5.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
test=pd.merge(test_id,agg_features_test,on='client_id')
test=pd.merge(test,cat_counts_test.reset_index(),on='client_id')

In [133]:
len(train)

30000

In [134]:
common_features=list(set(train.columns).intersection(set(test.columns)))
#common_features.remove('client_id')
common_features

['sum',
 'small_group_19',
 'small_group_67',
 'small_group_24',
 'small_group_177',
 'small_group_101',
 'small_group_135',
 'small_group_142',
 'small_group_171',
 'small_group_66',
 'small_group_86',
 'small_group_175',
 'small_group_98',
 'small_group_161',
 'max',
 'client_id',
 'small_group_154',
 'small_group_138',
 'small_group_147',
 'small_group_180',
 'small_group_72',
 'small_group_85',
 'small_group_140',
 'small_group_165',
 'small_group_157',
 'small_group_102',
 'small_group_129',
 'small_group_0',
 'small_group_50',
 'small_group_1',
 'small_group_148',
 'small_group_166',
 'small_group_118',
 'small_group_89',
 'small_group_6',
 'small_group_49',
 'small_group_81',
 'small_group_28',
 'small_group_91',
 'small_group_119',
 'small_group_8',
 'small_group_14',
 'small_group_159',
 'small_group_9',
 'small_group_97',
 'small_group_179',
 'small_group_187',
 'small_group_94',
 'small_group_90',
 'small_group_12',
 'small_group_18',
 'small_group_63',
 'small_group_117',
 

In [135]:
y_train_all=train['bins']
X_train_all=train[common_features]

#from sklearn.model_selection import train_test_split
#X_train,X_validation, y_train, y_validation = train_test_split(X_train_all, y_train_all, test_size=.33, random_state=1)

In [136]:
X_test=test[common_features]

In [137]:
#del all_tr, all_df

In [138]:
all_tr=pd.concat([transactions_train,transactions_test]).reset_index()
all_df=pd.concat([X_train_all, X_test]).reset_index()

In [139]:
all_tr

,index,client_id,trans_date,small_group,amount_rur,tr_month
0,0,33172,6,4,71.463,0
1,1,33172,6,35,45.017,0
2,2,33172,8,11,13.887,0
3,3,33172,9,11,15.983,0
4,4,33172,10,11,21.341,0
...,...,...,...,...,...,...
44117900,17667323,14864,727,1,21.824,24
44117901,17667324,14864,728,36,84.900,24
44117902,17667325,14864,728,3,2.748,24
44117903,17667326,14864,728,1,47.369,24


In [140]:
#all_tr['tr_month'] = (all_tr['trans_date']/30).astype('int')
#all_df['tr_month'] = (all_df['trans_date']/30).astype('int')

In [141]:
#aggregates shitstorm
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

from tqdm import tqdm

#step I - агрегаты по small_group
#for i in all_tr.small_group.unique():
#    temp = all_tr[all_tr.small_group==i]
#    temp = temp.groupby(['client_id'])['amount_rur'].agg(['sum','mean']).reset_index()
#    temp.columns = ['client_id','sum_sg{}'.format(i),'mean_sg{}'.format(i)]
#    all_df=pd.merge(all_df,temp,how='left',on='client_id')
#    all_df['mean_sg{}_frac'.format(i)] = all_df['mean_sg{}'.format(i)] / all_df['mean']
#    all_df['sum_sg{}_frac'.format(i)] = all_df['sum_sg{}'.format(i)] / all_df['sum']#было 0.57, стало 0,59

def pooled_func1(pooled_args):
    temp = all_tr[all_tr.small_group==pooled_args]
    temp = temp.groupby(['client_id'])['amount_rur'].agg(['sum','mean']).reset_index()
    temp.columns = ['client_id','sum_sg{}'.format(pooled_args),'mean_sg{}'.format(pooled_args)]
    return temp 
#multiprocessing
map_args = all_tr.small_group.unique()
with Pool(24) as pool:  # Create a multiprocessing Pool
    records = pool.map(pooled_func1, map_args)# process data_inputs iterable with pool
for i in range(len(records)):    
    all_df=pd.merge(all_df,records[i],how='left',on='client_id')
for i in map_args:
    all_df['mean_sg{}_frac'.format(i)] = all_df['mean_sg{}'.format(i)] / all_df['mean']
    all_df['sum_sg{}_frac'.format(i)] = all_df['sum_sg{}'.format(i)] / all_df['sum']#было 0.57, стало 0,59
    
    
    
#step II - агрегаты по месяцам
#for i in all_tr.tr_month.unique():
#    temp = all_tr[all_tr.tr_month==i]
#    temp = temp.groupby(['client_id'])['amount_rur'].agg(['sum','mean']).reset_index()
#    temp.columns = ['client_id','sum_mon{}'.format(i),'mean_mon{}'.format(i)]
#    all_df=pd.merge(all_df,temp,how='left',on='client_id')#было 0.59, стало хз, но 0.594

def pooled_func2(pooled_args):
    temp = all_tr[all_tr.tr_month==pooled_args]
    temp = temp.groupby(['client_id'])['amount_rur'].agg(['sum','mean']).reset_index()
    temp.columns = ['client_id','sum_mon{}'.format(pooled_args),'mean_mon{}'.format(pooled_args)]
    return temp 
#multiprocessing
map_args = all_tr.tr_month.unique()
with Pool(24) as pool:  # Create a multiprocessing Pool
    records = pool.map(pooled_func2, map_args)# process data_inputs iterable with pool
for i in range(len(records)):    
    all_df=pd.merge(all_df,records[i],how='left',on='client_id')#было 0.59, стало хз, но 0.594


all_df = all_df.fillna(0)
X_train_all = all_df[:len(X_train_all)]
X_test = all_df[len(X_train_all):]
del all_tr, all_df

In [142]:
#OOM if all small groups
from multiprocessing import Pool
from tqdm import tqdm

all_tr=pd.concat([transactions_train,transactions_test]).reset_index()
all_df=pd.concat([X_train_all, X_test]).reset_index()

def pooled_func3(pooled_args):
    temp0 = all_tr[all_tr.small_group==pooled_args]
    for i in temp0.tr_month.unique():
        temp = temp0[temp0.tr_month==i]
        temp = temp.groupby(['client_id',])['amount_rur'].agg(['sum','mean']).reset_index()
        temp.columns = ['client_id','sum_sg{}_m{}'.format(pooled_args,i),'mean_sg{}_m{}'.format(pooled_args,i)]
    return temp 

#multiprocessing
map_args = all_tr.small_group.unique()
with Pool(24) as pool:  # Create a multiprocessing Pool
    records = pool.map(pooled_func3, map_args)# process data_inputs iterable with pool
for i in range(len(records)):    
    all_df=pd.merge(all_df,records[i],how='left',on='client_id')

#for j in all_tr.small_group.unique():
#    temp0 = all_tr[all_tr.small_group==j]
#    for i in temp0.tr_month.unique():
#        temp = temp0[temp0.tr_month==i]
#        temp = temp.groupby(['client_id',])['amount_rur'].agg(['sum','mean']).reset_index()
#        temp.columns = ['client_id','sum_sg{}_m{}'.format(j,i),'mean_sg{}_m{}'.format(j,i)]
#        all_df=pd.merge(all_df,temp,how='left',on='client_id')
    
all_df = all_df.fillna(0)
X_train_all = all_df[:len(X_train_all)]
X_test = all_df[len(X_train_all):]
del all_tr, all_df

In [143]:
len(records)

204

In [144]:
print(X_train_all.columns)

Index(['level_0', 'index', 'sum', 'small_group_19', 'small_group_67',
       'small_group_24', 'small_group_177', 'small_group_101',
       'small_group_135', 'small_group_142',
       ...
       'sum_sg202_m7', 'mean_sg202_m7', 'sum_sg203_m6', 'mean_sg203_m6',
       'sum_sg188_m17', 'mean_sg188_m17', 'sum_sg194_m7', 'mean_sg194_m7',
       'sum_sg201_m4', 'mean_sg201_m4'],
      dtype='object', length=1482)


In [145]:
#all_tr=pd.concat([transactions_train,transactions_test]).reset_index()
#all_df=pd.concat([X_train_all, X_test]).reset_index()
#all_tr['client_mon_gr_mean'] = all_tr.groupby(['client_id','tr_month','small_group'])['amount_rur'].transform('mean')
#all_tr['client_mon_gr_sum'] = all_tr.groupby(['client_id','tr_month','small_group'])['amount_rur'].transform('sum')
#all_tr['client_mon_mean'] = all_tr.groupby(['client_id','tr_month'])['amount_rur'].transform('mean')
#all_tr['client_mon_sum'] = all_tr.groupby(['client_id','tr_month'])['amount_rur'].transform('sum')
#all_tr['client_gr_mean'] = all_tr.groupby(['client_id','small_group'])['amount_rur'].transform('mean')
#all_tr['client_gr_sum'] = all_tr.groupby(['client_id','small_group'])['amount_rur'].transform('sum')


In [146]:

#all_df=pd.merge(all_df,all_tr,how='left',on='client_id')
#all_df = all_df.fillna(0)
#X_train_all = all_df[:len(X_train_all)]
#X_test = all_df[len(X_train_all):]
#del all_tr, all_df

In [147]:
len(X_train_all)

30000

In [148]:
X_train_all = X_train_all.drop(['index','level_0'], axis=1)

In [149]:
X_train_all.head()

,sum,small_group_19,small_group_67,small_group_24,small_group_177,small_group_101,small_group_135,small_group_142,small_group_171,small_group_66,...,sum_sg202_m7,mean_sg202_m7,sum_sg203_m6,mean_sg203_m6,sum_sg188_m17,mean_sg188_m17,sum_sg194_m7,mean_sg194_m7,sum_sg201_m4,mean_sg201_m4
0,30254.011,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,42548.570,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26842.816,0.0,3.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15773.126,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12488.375,0.0,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


%%time
from sklearn.model_selection import train_test_split
X_train,X_validation, y_train, y_validation = train_test_split(X_train_all, y_train_all, test_size=.33, random_state=1)
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)

param = {'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':4,
    'n_estimators': 1200,
    'metric': 'multi_logloss',
    'n_jobs': -1,#new
    'verbose':-1,#new
    'learning_rate': 0.01,#0.005,
    'max_depth': 9,#50,
    'num_leaves': 2**8}
param = {'task': 'train',
        #'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class':4,
        #'n_estimators': 120,
        'metric': 'multi_logloss',
        'learning_rate': 0.005,
        'max_depth': 50,
        'num_leaves': 900,
        'feature_fraction': 0.4,
        'bagging_fraction': 0.6,
        'bagging_freq': 7}    

model = lgb.train(param, d_train,100)
from sklearn.metrics import accuracy_score

import numpy as np
print('Train Accuracy:', accuracy_score(y_train, [np.argmax(line) for line in model.predict(X_train)]))
print('Validation Accuracy:', accuracy_score(y_validation, [np.argmax(line) for line in model.predict(X_validation)]))

In [150]:
from sklearn.model_selection import train_test_split
X_train,X_validation, y_train, y_validation = train_test_split(X_train_all, y_train_all, test_size=.33, random_state=1)

In [151]:
%%time
#https://catboost.ai/docs/concepts/python-usages-examples.html
from catboost import Pool, CatBoostClassifier
train_dataset = Pool(data=X_train,
                      label=y_train,
                      ) 
eval_dataset = Pool(data=X_validation,
                     label=y_validation,
                     )
model = CatBoostClassifier( iterations=500,
                            one_hot_max_size=50,
                            l2_leaf_reg=9,
                            learning_rate=0.27,
                            depth=5,
                            #Train Accuracy: 0.7598507462686567
                            #Validation Accuracy: 0.6329292929292929
                            #iterations=500,
                            #one_hot_max_size=50,
                            #l2_leaf_reg=9,
                            #learning_rate=0.25,
                            #depth=5,
                            #0.6180:
                            ##num_trees=300,
                            ##learning_rate=0.35,
                            ##depth=3,    
                            loss_function='MultiClass')
model.fit(train_dataset,eval_set=eval_dataset)
#Get predicted classes
#preds_class = model.predict(eval_dataset)
#Get predicted probabilities for each class
#preds_proba = model.predict_proba(eval_dataset)
#Get predicted RawFormulaVal
#preds_raw = model.predict(eval_dataset, prediction_type='RawFormulaVal')


0:	learn: 1.2602757	test: 1.2601377	best: 1.2601377 (0)	total: 275ms	remaining: 2m 17s
1:	learn: 1.1832614	test: 1.1859868	best: 1.1859868 (1)	total: 540ms	remaining: 2m 14s
2:	learn: 1.1297528	test: 1.1336115	best: 1.1336115 (2)	total: 803ms	remaining: 2m 13s
3:	learn: 1.0879018	test: 1.0929423	best: 1.0929423 (3)	total: 1.06s	remaining: 2m 12s
4:	learn: 1.0545925	test: 1.0611971	best: 1.0611971 (4)	total: 1.33s	remaining: 2m 11s
5:	learn: 1.0318596	test: 1.0371571	best: 1.0371571 (5)	total: 1.6s	remaining: 2m 11s
6:	learn: 1.0111590	test: 1.0175271	best: 1.0175271 (6)	total: 1.86s	remaining: 2m 10s
7:	learn: 0.9974209	test: 1.0039051	best: 1.0039051 (7)	total: 2.12s	remaining: 2m 10s
8:	learn: 0.9843408	test: 0.9902819	best: 0.9902819 (8)	total: 2.38s	remaining: 2m 10s
9:	learn: 0.9731184	test: 0.9798934	best: 0.9798934 (9)	total: 2.65s	remaining: 2m 9s
10:	learn: 0.9656144	test: 0.9731271	best: 0.9731271 (10)	total: 2.91s	remaining: 2m 9s
11:	learn: 0.9565153	test: 0.9656080	best: 0

93:	learn: 0.7792374	test: 0.8501833	best: 0.8501833 (93)	total: 24.7s	remaining: 1m 46s
94:	learn: 0.7781497	test: 0.8498546	best: 0.8498546 (94)	total: 25s	remaining: 1m 46s
95:	learn: 0.7773459	test: 0.8496385	best: 0.8496385 (95)	total: 25.2s	remaining: 1m 46s
96:	learn: 0.7763410	test: 0.8492577	best: 0.8492577 (96)	total: 25.5s	remaining: 1m 45s
97:	learn: 0.7748085	test: 0.8485992	best: 0.8485992 (97)	total: 25.8s	remaining: 1m 45s
98:	learn: 0.7738040	test: 0.8480743	best: 0.8480743 (98)	total: 26s	remaining: 1m 45s
99:	learn: 0.7731941	test: 0.8476691	best: 0.8476691 (99)	total: 26.3s	remaining: 1m 45s
100:	learn: 0.7726814	test: 0.8477408	best: 0.8476691 (99)	total: 26.5s	remaining: 1m 44s
101:	learn: 0.7719074	test: 0.8471109	best: 0.8471109 (101)	total: 26.8s	remaining: 1m 44s
102:	learn: 0.7707950	test: 0.8465360	best: 0.8465360 (102)	total: 27.1s	remaining: 1m 44s
103:	learn: 0.7702496	test: 0.8464683	best: 0.8464683 (103)	total: 27.3s	remaining: 1m 44s
104:	learn: 0.7691

184:	learn: 0.7216973	test: 0.8393694	best: 0.8390377 (179)	total: 48.4s	remaining: 1m 22s
185:	learn: 0.7216170	test: 0.8393673	best: 0.8390377 (179)	total: 48.6s	remaining: 1m 22s
186:	learn: 0.7210661	test: 0.8390104	best: 0.8390104 (186)	total: 48.9s	remaining: 1m 21s
187:	learn: 0.7206415	test: 0.8389221	best: 0.8389221 (187)	total: 49.1s	remaining: 1m 21s
188:	learn: 0.7196900	test: 0.8385937	best: 0.8385937 (188)	total: 49.4s	remaining: 1m 21s
189:	learn: 0.7194577	test: 0.8384431	best: 0.8384431 (189)	total: 49.7s	remaining: 1m 21s
190:	learn: 0.7188168	test: 0.8383796	best: 0.8383796 (190)	total: 49.9s	remaining: 1m 20s
191:	learn: 0.7183385	test: 0.8381958	best: 0.8381958 (191)	total: 50.2s	remaining: 1m 20s
192:	learn: 0.7179032	test: 0.8381873	best: 0.8381873 (192)	total: 50.4s	remaining: 1m 20s
193:	learn: 0.7173361	test: 0.8381956	best: 0.8381873 (192)	total: 50.7s	remaining: 1m 19s
194:	learn: 0.7164676	test: 0.8383440	best: 0.8381873 (192)	total: 51s	remaining: 1m 19s
1

275:	learn: 0.6848831	test: 0.8346278	best: 0.8346278 (275)	total: 1m 11s	remaining: 58.4s
276:	learn: 0.6846499	test: 0.8345564	best: 0.8345564 (276)	total: 1m 12s	remaining: 58.1s
277:	learn: 0.6842655	test: 0.8344597	best: 0.8344597 (277)	total: 1m 12s	remaining: 57.8s
278:	learn: 0.6834710	test: 0.8344478	best: 0.8344478 (278)	total: 1m 12s	remaining: 57.6s
279:	learn: 0.6830511	test: 0.8346518	best: 0.8344478 (278)	total: 1m 12s	remaining: 57.3s
280:	learn: 0.6829331	test: 0.8346112	best: 0.8344478 (278)	total: 1m 13s	remaining: 57s
281:	learn: 0.6823428	test: 0.8345155	best: 0.8344478 (278)	total: 1m 13s	remaining: 56.8s
282:	learn: 0.6822864	test: 0.8345541	best: 0.8344478 (278)	total: 1m 13s	remaining: 56.5s
283:	learn: 0.6819717	test: 0.8345035	best: 0.8344478 (278)	total: 1m 13s	remaining: 56.3s
284:	learn: 0.6814631	test: 0.8347072	best: 0.8344478 (278)	total: 1m 14s	remaining: 56s
285:	learn: 0.6809204	test: 0.8347059	best: 0.8344478 (278)	total: 1m 14s	remaining: 55.8s
286

366:	learn: 0.6564656	test: 0.8316530	best: 0.8316530 (366)	total: 1m 35s	remaining: 34.7s
367:	learn: 0.6560489	test: 0.8317703	best: 0.8316530 (366)	total: 1m 36s	remaining: 34.4s
368:	learn: 0.6554612	test: 0.8317366	best: 0.8316530 (366)	total: 1m 36s	remaining: 34.2s
369:	learn: 0.6554348	test: 0.8317361	best: 0.8316530 (366)	total: 1m 36s	remaining: 33.9s
370:	learn: 0.6550210	test: 0.8317376	best: 0.8316530 (366)	total: 1m 36s	remaining: 33.7s
371:	learn: 0.6544227	test: 0.8314699	best: 0.8314699 (371)	total: 1m 37s	remaining: 33.4s
372:	learn: 0.6539874	test: 0.8313319	best: 0.8313319 (372)	total: 1m 37s	remaining: 33.1s
373:	learn: 0.6534647	test: 0.8315025	best: 0.8313319 (372)	total: 1m 37s	remaining: 32.9s
374:	learn: 0.6533469	test: 0.8315061	best: 0.8313319 (372)	total: 1m 37s	remaining: 32.6s
375:	learn: 0.6533268	test: 0.8315125	best: 0.8313319 (372)	total: 1m 38s	remaining: 32.4s
376:	learn: 0.6532755	test: 0.8314655	best: 0.8313319 (372)	total: 1m 38s	remaining: 32.1s

457:	learn: 0.6316672	test: 0.8304667	best: 0.8303403 (450)	total: 1m 59s	remaining: 11s
458:	learn: 0.6316516	test: 0.8304699	best: 0.8303403 (450)	total: 1m 59s	remaining: 10.7s
459:	learn: 0.6316363	test: 0.8304676	best: 0.8303403 (450)	total: 2m	remaining: 10.5s
460:	learn: 0.6316162	test: 0.8304611	best: 0.8303403 (450)	total: 2m	remaining: 10.2s
461:	learn: 0.6313873	test: 0.8304112	best: 0.8303403 (450)	total: 2m	remaining: 9.93s
462:	learn: 0.6310493	test: 0.8302938	best: 0.8302938 (462)	total: 2m	remaining: 9.67s
463:	learn: 0.6306571	test: 0.8302410	best: 0.8302410 (463)	total: 2m 1s	remaining: 9.41s
464:	learn: 0.6306395	test: 0.8302398	best: 0.8302398 (464)	total: 2m 1s	remaining: 9.15s
465:	learn: 0.6302985	test: 0.8303287	best: 0.8302398 (464)	total: 2m 1s	remaining: 8.88s
466:	learn: 0.6302855	test: 0.8303235	best: 0.8302398 (464)	total: 2m 2s	remaining: 8.62s
467:	learn: 0.6298590	test: 0.8305511	best: 0.8302398 (464)	total: 2m 2s	remaining: 8.36s
468:	learn: 0.6296392	

In [152]:
#### accuracy_score for catboost
from sklearn.metrics import accuracy_score
print('Train Accuracy:', accuracy_score(y_train, model.predict(X_train)))
print('Validation Accuracy:', accuracy_score(y_validation, model.predict(X_validation)))

Train Accuracy: 0.7623383084577114
Validation Accuracy: 0.6319191919191919


In [59]:
train_all_dataset = Pool(data=X_train_all,
                      label=y_train_all,
                      ) 
model.fit(train_all_dataset)


0:	learn: 1.2539868	total: 93.6ms	remaining: 46.7s
1:	learn: 1.1748964	total: 189ms	remaining: 47s
2:	learn: 1.1208249	total: 273ms	remaining: 45.3s
3:	learn: 1.0833978	total: 358ms	remaining: 44.4s
4:	learn: 1.0505581	total: 442ms	remaining: 43.8s
5:	learn: 1.0272168	total: 528ms	remaining: 43.5s
6:	learn: 1.0062874	total: 614ms	remaining: 43.3s
7:	learn: 0.9911016	total: 700ms	remaining: 43s
8:	learn: 0.9786690	total: 786ms	remaining: 42.9s
9:	learn: 0.9690213	total: 872ms	remaining: 42.7s
10:	learn: 0.9595446	total: 955ms	remaining: 42.5s
11:	learn: 0.9516161	total: 1.04s	remaining: 42.3s
12:	learn: 0.9420583	total: 1.12s	remaining: 42s
13:	learn: 0.9340757	total: 1.21s	remaining: 41.9s
14:	learn: 0.9290011	total: 1.29s	remaining: 41.6s
15:	learn: 0.9252097	total: 1.36s	remaining: 41.3s
16:	learn: 0.9188301	total: 1.45s	remaining: 41.2s
17:	learn: 0.9145682	total: 1.53s	remaining: 41s
18:	learn: 0.9103135	total: 1.61s	remaining: 40.9s
19:	learn: 0.9063004	total: 1.7s	remaining: 40.7

162:	learn: 0.7484870	total: 13.1s	remaining: 27s
163:	learn: 0.7479374	total: 13.1s	remaining: 26.9s
164:	learn: 0.7475046	total: 13.2s	remaining: 26.8s
165:	learn: 0.7470015	total: 13.3s	remaining: 26.8s
166:	learn: 0.7469809	total: 13.4s	remaining: 26.7s
167:	learn: 0.7465856	total: 13.5s	remaining: 26.6s
168:	learn: 0.7463732	total: 13.5s	remaining: 26.5s
169:	learn: 0.7459203	total: 13.6s	remaining: 26.4s
170:	learn: 0.7455702	total: 13.7s	remaining: 26.3s
171:	learn: 0.7449264	total: 13.8s	remaining: 26.3s
172:	learn: 0.7444873	total: 13.8s	remaining: 26.2s
173:	learn: 0.7440847	total: 13.9s	remaining: 26.1s
174:	learn: 0.7436702	total: 14s	remaining: 26s
175:	learn: 0.7433509	total: 14.1s	remaining: 25.9s
176:	learn: 0.7425830	total: 14.2s	remaining: 25.8s
177:	learn: 0.7422884	total: 14.2s	remaining: 25.8s
178:	learn: 0.7415327	total: 14.3s	remaining: 25.7s
179:	learn: 0.7414728	total: 14.4s	remaining: 25.6s
180:	learn: 0.7410769	total: 14.5s	remaining: 25.5s
181:	learn: 0.7409

321:	learn: 0.6965818	total: 25.4s	remaining: 14s
322:	learn: 0.6962781	total: 25.5s	remaining: 14s
323:	learn: 0.6959051	total: 25.6s	remaining: 13.9s
324:	learn: 0.6958740	total: 25.6s	remaining: 13.8s
325:	learn: 0.6955347	total: 25.7s	remaining: 13.7s
326:	learn: 0.6950874	total: 25.8s	remaining: 13.6s
327:	learn: 0.6946830	total: 25.9s	remaining: 13.6s
328:	learn: 0.6946564	total: 25.9s	remaining: 13.5s
329:	learn: 0.6945305	total: 26s	remaining: 13.4s
330:	learn: 0.6944921	total: 26.1s	remaining: 13.3s
331:	learn: 0.6943008	total: 26.2s	remaining: 13.2s
332:	learn: 0.6936960	total: 26.2s	remaining: 13.2s
333:	learn: 0.6933951	total: 26.3s	remaining: 13.1s
334:	learn: 0.6933543	total: 26.4s	remaining: 13s
335:	learn: 0.6933393	total: 26.5s	remaining: 12.9s
336:	learn: 0.6933128	total: 26.5s	remaining: 12.8s
337:	learn: 0.6932543	total: 26.6s	remaining: 12.8s
338:	learn: 0.6928766	total: 26.7s	remaining: 12.7s
339:	learn: 0.6926413	total: 26.8s	remaining: 12.6s
340:	learn: 0.691986

480:	learn: 0.6618327	total: 37.6s	remaining: 1.49s
481:	learn: 0.6616311	total: 37.7s	remaining: 1.41s
482:	learn: 0.6611725	total: 37.8s	remaining: 1.33s
483:	learn: 0.6608261	total: 37.8s	remaining: 1.25s
484:	learn: 0.6605492	total: 37.9s	remaining: 1.17s
485:	learn: 0.6601560	total: 38s	remaining: 1.09s
486:	learn: 0.6598928	total: 38.1s	remaining: 1.02s
487:	learn: 0.6594732	total: 38.2s	remaining: 938ms
488:	learn: 0.6593289	total: 38.2s	remaining: 860ms
489:	learn: 0.6593178	total: 38.3s	remaining: 782ms
490:	learn: 0.6593102	total: 38.4s	remaining: 704ms
491:	learn: 0.6592960	total: 38.5s	remaining: 625ms
492:	learn: 0.6588380	total: 38.5s	remaining: 547ms
493:	learn: 0.6586054	total: 38.6s	remaining: 469ms
494:	learn: 0.6584031	total: 38.7s	remaining: 391ms
495:	learn: 0.6577137	total: 38.8s	remaining: 313ms
496:	learn: 0.6573775	total: 38.8s	remaining: 235ms
497:	learn: 0.6570338	total: 38.9s	remaining: 156ms
498:	learn: 0.6568443	total: 39s	remaining: 78.2ms
499:	learn: 0.6

In [58]:
pred = model.predict(X_test).flatten().astype(int)

In [59]:
pred

array([0, 2, 0, ..., 2, 2, 3])

In [60]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,0
19974,2
10505,1


In [61]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

submissions/1578474546.csv


from collections import Counter
def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: round(float(majority)/float(count), 5) for cls, count in counter.items()}

class_weights = get_class_weights(y_train_all.values)
print(class_weights)

#%%time
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
folds = KFold(n_splits=5)

param = {'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':4,
    'n_estimators': 1200,
    'metric': 'multi_logloss',
    'n_jobs': -1,#new
    'verbose':-1,#new
    'learning_rate': 0.01,#0.005,
    'max_depth': 9,#50,
    'num_leaves': 2**8}

scores = list()
train_acc = list()
val_acc = list()
feature_importances = pd.DataFrame()
feature_importances['feature'] = X_train_all.columns

#training_start_time = time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X_train_all, y_train_all)):
    #start_time = time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X_train_all.iloc[trn_idx], label=y_train_all.iloc[trn_idx])
    val_data = lgb.Dataset(X_train_all.iloc[test_idx], label=y_train_all.iloc[test_idx])
    clf = lgb.train(param, trn_data, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=100)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    scores.append(clf.best_score['valid_1']['multi_logloss'])
    train_acc.append(accuracy_score(y_train_all.iloc[trn_idx], [np.argmax(line) for line in clf.predict(X_train_all.iloc[trn_idx])]))
    val_acc.append(accuracy_score(y_train_all.iloc[test_idx], [np.argmax(line) for line in clf.predict(X_train_all.iloc[test_idx])]))
    
    #print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
print('-' * 30)
print('Training has finished.')
#print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
print('Mean AUC:', np.mean(scores))
print('-' * 30)
print('Mean train ACC:', np.mean(train_acc))
print('Mean val ACC:', np.mean(val_acc))

print('-' * 30)
print('Max train ACC:', np.max(train_acc))
print('Max val ACC:', np.max(val_acc))

import matplotlib.pyplot as plt
import seaborn as sns
feature_importances['average'] = feature_importances[['fold_{}'.format(fold + 1) for fold in range(folds.n_splits)]].mean(axis=1)
#feature_importances.to_csv('feature_importances.csv')

plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(50), x='average', y='feature');
plt.title('50 TOP feature importance over {} folds average'.format(folds.n_splits));